In [1]:
import os
import random
import numpy as np
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.config import get_cfg
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.model_zoo import model_zoo
from detectron2.utils.visualizer import Visualizer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

# Register your training dataset with Detectron2
register_coco_instances("custom_dataset_train", {}, "train_output_coco_annotations.json", "train/images")

# Register your test dataset with Detectron2
register_coco_instances("custom_dataset_test", {}, "test_output_coco_annotations.json", "test/images")

# Define metadata for your training dataset (class names)
metadata_train = MetadataCatalog.get("custom_dataset_train")

# Define metadata for your test dataset (class names)
metadata_test = MetadataCatalog.get("custom_dataset_test")

# Create a configuration
cfg = get_cfg()

# Set your custom configuration options here, for example:
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.DATASETS.TRAIN = ("custom_dataset_train",)
cfg.DATASETS.TEST = ("custom_dataset_test",)  # Include your test dataset
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 1111  # Increase the maximum number of iterations
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.DEVICE = 'cpu'  # Use 'cuda' if GPU is available

# Instantiate a trainer
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)

# Train the model
trainer.train()

# Optionally, evaluate the model on the test set
evaluator = COCOEvaluator("custom_dataset_test", cfg, False, output_dir="./output/")
test_loader = build_detection_test_loader(cfg, "custom_dataset_test")
test_results = inference_on_dataset(trainer.model, test_loader, evaluator)

# Save the model's configuration to a YAML file
model_config_path = "model_config.yaml"
with open(model_config_path, "w") as f:
    f.write(cfg.dump())

print(f"Model's configuration saved to {model_config_path}")

# Print and visualize the test results
print("Test results:")
print(test_results)


[09/09 00:59:46 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

[09/09 00:59:47 d2.data.datasets.coco]: Loaded 5690 images in COCO format from train_output_coco_annotations.json
[09/09 00:59:47 d2.data.build]: Removed 4198 images with no usable annotations. 1492 images left.
[09/09 00:59:47 d2.data.build]: Distribution of instances among all 1 categories:
|  category  | #instances   |
|:----------:|:-------------|
|   damage   | 32467        |
|            |              |
[09/09 00:59:47 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in training: [ResizeShortestEdge(short_edge_length=(640, 672, 704, 736, 768, 800), max_size=1333, sample_style='choice'), RandomFlip()]
[09/09 00:59:47 d2.data.build]: Using training sampler TrainingSampler
[09/09 00:59:47 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[09/09 00:59:47 d2.data.common]: Serializing 1492 elements to byte tensors and concatenating them all ...
[09/09 00:59:47 d2.data.common]: Serialized dataset takes 10.33 MiB
WARNING [09

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (1, 256, 1, 1) in

[09/09 00:59:47 d2.engine.train_loop]: Starting training from iteration 0


/Users/dhruvraj/venv-metal/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


...

[09/09 02:28:20 d2.evaluation.evaluator]: Inference done 866/1866. Dataloading: 0.0003 s/iter. Inference: 0.9639 s/iter. Eval: 0.1079 s/iter. Total: 1.0723 s/iter. ETA=0:17:52
[09/09 02:28:25 d2.evaluation.evaluator]: Inference done 871/1866. Dataloading: 0.0003 s/iter. Inference: 0.9638 s/iter. Eval: 0.1079 s/iter. Total: 1.0723 s/iter. ETA=0:17:46
[09/09 02:28:31 d2.evaluation.evaluator]: Inference done 876/1866. Dataloading: 0.0003 s/iter. Inference: 0.9639 s/iter. Eval: 0.1079 s/iter. Total: 1.0723 s/iter. ETA=0:17:41
[09/09 02:28:36 d2.evaluation.evaluator]: Inference done 881/1866. Dataloading: 0.0003 s/iter. Inference: 0.9638 s/iter. Eval: 0.1079 s/iter. Total: 1.0723 s/iter. ETA=0:17:36
[09/09 02:28:41 d2.evaluation.evaluator]: Inference done 886/1866. Dataloading: 0.0003 s/iter. Inference: 0.9638 s/iter. Eval: 0.1079 s/iter. Total: 1.0722 s/iter. ETA=0:17:30
[09/09 02:28:47 d2.evaluation.evaluator]: Inference done 891/1866. Dataloading: 0.0003 s/iter. Inference: 0.9639 s/iter.

[09/09 02:32:23 d2.evaluation.evaluator]: Inference done 1091/1866. Dataloading: 0.0003 s/iter. Inference: 0.9653 s/iter. Eval: 0.1080 s/iter. Total: 1.0738 s/iter. ETA=0:13:52
[09/09 02:32:28 d2.evaluation.evaluator]: Inference done 1096/1866. Dataloading: 0.0003 s/iter. Inference: 0.9653 s/iter. Eval: 0.1080 s/iter. Total: 1.0738 s/iter. ETA=0:13:46
[09/09 02:32:33 d2.evaluation.evaluator]: Inference done 1101/1866. Dataloading: 0.0003 s/iter. Inference: 0.9653 s/iter. Eval: 0.1080 s/iter. Total: 1.0738 s/iter. ETA=0:13:41
[09/09 02:32:39 d2.evaluation.evaluator]: Inference done 1106/1866. Dataloading: 0.0003 s/iter. Inference: 0.9652 s/iter. Eval: 0.1080 s/iter. Total: 1.0737 s/iter. ETA=0:13:36
[09/09 02:32:44 d2.evaluation.evaluator]: Inference done 1111/1866. Dataloading: 0.0003 s/iter. Inference: 0.9652 s/iter. Eval: 0.1080 s/iter. Total: 1.0737 s/iter. ETA=0:13:30
[09/09 02:32:49 d2.evaluation.evaluator]: Inference done 1116/1866. Dataloading: 0.0003 s/iter. Inference: 0.9652 s

[09/09 02:36:26 d2.evaluation.evaluator]: Inference done 1316/1866. Dataloading: 0.0003 s/iter. Inference: 0.9662 s/iter. Eval: 0.1082 s/iter. Total: 1.0750 s/iter. ETA=0:09:51
[09/09 02:36:31 d2.evaluation.evaluator]: Inference done 1321/1866. Dataloading: 0.0003 s/iter. Inference: 0.9663 s/iter. Eval: 0.1082 s/iter. Total: 1.0750 s/iter. ETA=0:09:45
[09/09 02:36:36 d2.evaluation.evaluator]: Inference done 1326/1866. Dataloading: 0.0003 s/iter. Inference: 0.9663 s/iter. Eval: 0.1082 s/iter. Total: 1.0750 s/iter. ETA=0:09:40
[09/09 02:36:42 d2.evaluation.evaluator]: Inference done 1331/1866. Dataloading: 0.0003 s/iter. Inference: 0.9664 s/iter. Eval: 0.1082 s/iter. Total: 1.0751 s/iter. ETA=0:09:35
[09/09 02:36:47 d2.evaluation.evaluator]: Inference done 1336/1866. Dataloading: 0.0003 s/iter. Inference: 0.9665 s/iter. Eval: 0.1082 s/iter. Total: 1.0752 s/iter. ETA=0:09:29
[09/09 02:36:53 d2.evaluation.evaluator]: Inference done 1341/1866. Dataloading: 0.0003 s/iter. Inference: 0.9664 s

[09/09 02:40:25 d2.evaluation.evaluator]: Inference done 1541/1866. Dataloading: 0.0003 s/iter. Inference: 0.9650 s/iter. Eval: 0.1082 s/iter. Total: 1.0737 s/iter. ETA=0:05:48
[09/09 02:40:31 d2.evaluation.evaluator]: Inference done 1546/1866. Dataloading: 0.0003 s/iter. Inference: 0.9649 s/iter. Eval: 0.1082 s/iter. Total: 1.0737 s/iter. ETA=0:05:43
[09/09 02:40:36 d2.evaluation.evaluator]: Inference done 1551/1866. Dataloading: 0.0003 s/iter. Inference: 0.9649 s/iter. Eval: 0.1082 s/iter. Total: 1.0736 s/iter. ETA=0:05:38
[09/09 02:40:41 d2.evaluation.evaluator]: Inference done 1556/1866. Dataloading: 0.0003 s/iter. Inference: 0.9649 s/iter. Eval: 0.1082 s/iter. Total: 1.0736 s/iter. ETA=0:05:32
[09/09 02:40:47 d2.evaluation.evaluator]: Inference done 1561/1866. Dataloading: 0.0003 s/iter. Inference: 0.9648 s/iter. Eval: 0.1082 s/iter. Total: 1.0735 s/iter. ETA=0:05:27
[09/09 02:40:52 d2.evaluation.evaluator]: Inference done 1566/1866. Dataloading: 0.0003 s/iter. Inference: 0.9647 s

[09/09 02:44:24 d2.evaluation.evaluator]: Inference done 1766/1866. Dataloading: 0.0003 s/iter. Inference: 0.9633 s/iter. Eval: 0.1081 s/iter. Total: 1.0719 s/iter. ETA=0:01:47
[09/09 02:44:29 d2.evaluation.evaluator]: Inference done 1771/1866. Dataloading: 0.0003 s/iter. Inference: 0.9633 s/iter. Eval: 0.1081 s/iter. Total: 1.0719 s/iter. ETA=0:01:41
[09/09 02:44:35 d2.evaluation.evaluator]: Inference done 1776/1866. Dataloading: 0.0003 s/iter. Inference: 0.9633 s/iter. Eval: 0.1081 s/iter. Total: 1.0719 s/iter. ETA=0:01:36
[09/09 02:44:40 d2.evaluation.evaluator]: Inference done 1781/1866. Dataloading: 0.0003 s/iter. Inference: 0.9633 s/iter. Eval: 0.1081 s/iter. Total: 1.0719 s/iter. ETA=0:01:31
[09/09 02:44:45 d2.evaluation.evaluator]: Inference done 1786/1866. Dataloading: 0.0003 s/iter. Inference: 0.9633 s/iter. Eval: 0.1081 s/iter. Total: 1.0719 s/iter. ETA=0:01:25
[09/09 02:44:51 d2.evaluation.evaluator]: Inference done 1791/1866. Dataloading: 0.0003 s/iter. Inference: 0.9633 s

In [3]:
import os
import random
import numpy as np
from detectron2.data import MetadataCatalog
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
import cv2

# Load the saved model weights and configuration
cfg = get_cfg()
cfg.merge_from_file("model_config.yaml")  # Replace with the path to your model's configuration file
cfg.MODEL.WEIGHTS = "output/model_final.pth"  # Replace with the path to your saved model weights

# Set the device to CPU
cfg.MODEL.DEVICE = 'cpu'

# Create a predictor using the loaded model
predictor = DefaultPredictor(cfg)

# Get a random image from the "train/images" folder
image_folder = 'train/images'  # Replace with the path to your image folder
image_files = os.listdir(image_folder)
random_image_file = random.choice(image_files)
image_path = os.path.join(image_folder, random_image_file)

# Read the random image
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Get predictions on the image
outputs = predictor(image)

# Filter predictions with confidence > 75%
instances = outputs["instances"]
filtered_instances = instances[instances.scores > 0.75]

# Visualize the original image with filtered predictions
v = Visualizer(image, metadata=MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(filtered_instances.to("cpu"))

# Show the image with predictions
cv2.imshow("Predictions", out.get_image()[:, :, ::-1])

# Construct the path to the corresponding mask image
mask_folder = 'train/binned_targets'  # Replace with the path to your mask folder
mask_file = os.path.join(mask_folder, os.path.splitext(random_image_file)[0] + '_target.png')

# Read and show the mask image
mask_image = cv2.imread(mask_file)
cv2.imshow("Mask", mask_image)

cv2.waitKey(0)
# Close all OpenCV windows when any key is pressed
cv2.destroyAllWindows()
cv2.waitKey(1)

-1